# header

In [19]:
import os
import json
import datetime
import logging
import warnings

import pandas as pd
import numpy as np
from tqdm import tqdm
from numba import cuda
from collections import Counter

from sklearn.metrics import accuracy_score, precision_score, recall_score
import config as CONFIG
import utils

import wandb

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, Dataset, SubsetRandomSampler
from sklearn.metrics import precision_score, recall_score
import torch.nn as nn
import torch.nn.functional as F


# generate features

In [23]:
# single process, go "feats.py" for multi process
max_row = 131072
max_col = 1024
row_block = 32
col_block = 16
row_section_size = max_row // row_block
col_section_size = max_col // col_block

def process_one(i):
    df = pd.read_csv(os.path.join(CONFIG.PATH_PROCESSED, f"mcelog_{i}.csv"))
    data = []
    labels = []
    hosts = df["sid"].drop_duplicates().to_list()

    for host in tqdm(hosts):
        host_matrix = np.zeros([row_block, col_block])
        host_df = df[df["sid"]==host]
        host_df = host_df.fillna(-1)
        
        label = (int(host_df["failure_type"].sum() < 0) + 1) % 2
        labels.append(label)

        host_df["row_section"] = host_df["row"] // row_section_size
        host_df["col_section"] = host_df["col"] // col_section_size

        host_df_simple = host_df[["row_section", "col_section"]].drop_duplicates()

        for _, row in host_df_simple.iterrows():
            host_matrix[int(row["row_section"]), int(row["col_section"])] = 1
        data.append(host_matrix)
    
    np.save(os.path.join(CONFIG.PATH_PROCESSED, f"feats_{i}.npy"), data)
    np.save(os.path.join(CONFIG.PATH_PROCESSED, f"labels_{i}.npy"), labels)
    
    
# for i in range(0, 31):
#     if i == 22:
#         continue
#     process_one(i)

# load data

In [41]:
host_matrix = np.zeros([3, 3])

host_matrix[1,1] += 1
host_matrix[1,1] += 1.2

host_matrix

array([[0. , 0. , 0. ],
       [0. , 2.2, 0. ],
       [0. , 0. , 0. ]])

In [16]:

dataset_list = []


for i in range(31):
    if i == 22:
        continue
    data_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"feats_64x32_with_times_{i}.npy"))
    label_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"labels_64x32_{i}.npy"))
    
    data_np = 2 / (1 + np.exp(-data_np)) - 1

    data = torch.tensor(data_np, dtype=torch.float32)
    label = torch.tensor(label_np, dtype=torch.int64)
     
    dataset = TensorDataset(data, label)
    dataset_list.append(dataset)

datasets = torch.utils.data.ConcatDataset(dataset_list)

In [17]:
data_np[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## check ratio

In [48]:

dataset_list = []

xgb_data_list = []
xgb_label_list = []

cnt = 0
error = 0
enrich_ratio = 1
under_sampling_ratio = 8

for i in range(31):
    if i == 22:
        continue
    data_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"feats_64x32_with_times_{i}.npy"))
    label_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"labels_64x32_{i}.npy"))
    
    data_list = data_np.tolist()
    label_list = label_np.tolist()
    
    tmp_data_list = []
    tmp_label_list = []
    for idx in range(len(data_list)):
        curdata = data_list[idx]
        curlabel = label_list[idx]
        if curlabel == 1:
            for _ in range(enrich_ratio):
                tmp_data_list.append(curdata)
                tmp_label_list.append(curlabel)
        else:
            if idx % under_sampling_ratio == 0:
                tmp_data_list.append(curdata)
                tmp_label_list.append(curlabel)
    cnt += len(label_list)   
    error += sum(label_list)
    
    
    data_list = tmp_data_list
    label_list = tmp_label_list
    
    xgb_data_list.extend(data_list)
    xgb_label_list.extend(label_list)
    
    data_np = np.array(data_list)
    label_np = np.array(label_list)
    
    data_np = 2 / (1 + np.exp(-data_np)) - 1
    
    data = torch.tensor(data_np, dtype=torch.float32)
    label = torch.tensor(label_np, dtype=torch.int64)
     
    dataset = TensorDataset(data, label)
    dataset_list.append(dataset)

datasets = torch.utils.data.ConcatDataset(dataset_list)

In [49]:
len(xgb_data_list)

4296

In [50]:
sum(xgb_label_list)

566

In [51]:
data.shape

torch.Size([2, 1, 64, 32])

In [23]:
data_np.shape

(2, 1, 64, 32)

In [30]:
datasets[0].len()

AttributeError: 'tuple' object has no attribute 'len'

In [59]:
ret_data = None
ret_label = None

for i in range(31):
    if i == 22:
        continue
    data_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"feats_32x16_with_times_{i}.npy"))
    label_np = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"labels_32x16_{i}.npy"))
    if ret_data is None:
        ret_data = data_np
        ret_label = label_np
    else:
        ret_data = np.concatenate((ret_data, data_np))
        ret_label = np.concatenate((ret_label, label_np))

In [60]:
ret_data.shape

(30264, 1, 32, 16)

In [61]:
ret_label.shape

(30264,)

In [62]:
ret_label[256]

0

In [63]:
np.save(os.path.join(CONFIG.PATH_PROCESSED, "feats_32x16_with_times.npy"), ret_data)
np.save(os.path.join(CONFIG.PATH_PROCESSED, "labels_32x16.npy"), ret_label)    

In [85]:
ddd = np.load(os.path.join(CONFIG.PATH_PROCESSED, "feats_32x16_with_times.npy"))
lll = np.load(os.path.join(CONFIG.PATH_PROCESSED, "labels_32x16.npy"))    

In [87]:
ddd.shape

(30264, 1, 32, 16)

In [88]:
ddd = np.load(os.path.join(CONFIG.PATH_PROCESSED, "feats_64x32_with_times.npy"))
lll = np.load(os.path.join(CONFIG.PATH_PROCESSED, "labels_64x32.npy"))    

In [91]:
ddd.shape

(30264, 1, 64, 32)

In [95]:
ddd[345].any()

True

# model

## single testing

In [56]:
torch.manual_seed(42)
np.random.seed(42)

import torch.nn as nn
import torch.nn.functional as F
import random
random.seed(42)

# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
#         self.channels = 64
        
#         self.conv1 = nn.Conv2d(1, self.channels, kernel_size=9, padding=4)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(self.channels * 16 * 8, 128)  
#         self.dropout = nn.Dropout(0.1)
#         self.fc2 = nn.Linear(128, 2)  

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = x.view(-1, self.channels * 16 * 8)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
#         self.channels1 = 32
#         self.channels2 = 64
        
#         self.conv1 = nn.Conv2d(1, self.channels1, kernel_size=3, padding=1)
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         self.conv2 = nn.Conv2d(self.channels1, self.channels2, kernel_size=3, padding=1)
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         self.fc1 = nn.Linear(self.channels2 * 7 * 7, 128)
        
#         self.fc2 = nn.Linear(128, 2)  # 输出2类，根据你的任务调整输出维度

#         self.dropout = nn.Dropout(0.1)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = x.view(-1, self.channels2 * 7 * 7)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

# import torch
# import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=9, padding=4)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, padding=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        # self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        # self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        # 全连接层
        self.fc1 = nn.Linear(256 * 2 * 1, 256) 
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(256, 2)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 256 * 2 * 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

########################################################################
# 定义基本的ResNet块
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=9, stride=stride, padding=4, bias=False)
        # self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        # out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        # out = self.bn2(out)
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

# 定义ResNet网络
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 创建ResNet模型
def ResNetCustom():
    return ResNet(BasicBlock, [2, 2, 2])  # 这里定义了一个3层的ResNet
########################################################################

# wandb.init(
#     project="dram",
    
#     config={

#     }
# )
    
    

batch_size = 64
data_loader = DataLoader(datasets, shuffle=True)


train_ratio = 0.8
test_ratio = 1 - train_ratio  

total_samples = len(data_loader.dataset)

train_size = int(train_ratio * total_samples)
test_size = total_samples - train_size

indices = list(range(total_samples))

random.shuffle(indices)

train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(datasets, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(datasets, batch_size=batch_size, sampler=test_sampler)


############################################################################



#######################################################################################

# # model = CNNModel()
# model = ResNetCustom()


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("当前设备:", device)
# model.to(device)

# criterion = nn.CrossEntropyLoss() 
# optimizer = optim.Adam(model.parameters(), lr=0.001) 

# num_epochs = 100
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for _inputs, _labels in train_loader:
#         _inputs, _labels = _inputs.to(device), _labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(_inputs.unsqueeze(1))  # 在通道维度上增加1
#         loss = criterion(outputs, _labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         # wandb.log({"loss": running_loss})
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(data_loader)}')

# print('Finished Training')


# model.eval()



# predictions = []
# true_labels = []

# with torch.no_grad():
#     for _inputs, _labels in train_loader:
#         _inputs, _labels = _inputs.to(device), _labels.to(device)
#         outputs = model(_inputs.unsqueeze(1))
#         _, predicted = torch.max(outputs.data, 1)
#         predictions.extend(predicted.tolist())
#         true_labels.extend(_labels.tolist())

# precision = precision_score(true_labels, predictions)
# recall = recall_score(true_labels, predictions)

# print(f"all: {len(true_labels)}, error_labels: {sum(true_labels)}")
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')

# # wandb.log({"train precision": precision, "train recall": recall})


# predictions = []
# true_labels = []

# with torch.no_grad():
#     for _inputs, _labels in test_loader:
#         _inputs, _labels = _inputs.to(device), _labels.to(device)
#         outputs = model(_inputs.unsqueeze(1))
#         _, predicted = torch.max(outputs.data, 1)
#         predictions.extend(predicted.tolist())
#         true_labels.extend(_labels.tolist())

# print(f"all: {len(true_labels)}, error_labels: {sum(true_labels)}")
        
# precision = precision_score(true_labels, predictions)
# recall = recall_score(true_labels, predictions)

# print(f'Precision: {precision}')
# print(f'Recall: {recall}')

# # wandb.log({"test precision": precision, "test recall": recall})

# # wandb.finish()
#######################################################################################


model = CNNModel()
# wandb.watch(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("当前设备:", device)
model.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 

num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for _inputs, _labels in train_loader:
        _inputs, _labels = _inputs.to(device), _labels.to(device)
        optimizer.zero_grad()
        outputs = model(_inputs)  # 在通道维度上增加1
        loss = criterion(outputs, _labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # wandb.log({"loss": running_loss})
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(data_loader)}')

print('Finished Training')


model.eval()



predictions = []
true_labels = []

with torch.no_grad():
    for _inputs, _labels in train_loader:
        _inputs, _labels = _inputs.to(device), _labels.to(device)
        outputs = model(_inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(_labels.tolist())

precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f"all: {len(true_labels)}, error_labels: {sum(true_labels)}")
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# wandb.log({"train precision": precision, "train recall": recall})


predictions = []
true_labels = []

with torch.no_grad():
    for _inputs, _labels in test_loader:
        _inputs, _labels = _inputs.to(device), _labels.to(device)
        outputs = model(_inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())
        true_labels.extend(_labels.tolist())

print(f"all: {len(true_labels)}, error_labels: {sum(true_labels)}")
        
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

# wandb.log({"test precision": precision, "test recall": recall})

# wandb.finish()

当前设备: cuda


ValueError: Expected input batch_size (256) to match target batch_size (64).

## lightgbm

In [54]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假设你有N个样本，每个样本是一个3x2的矩阵
# N = 1000
data = np.array(xgb_data_list)
labels = np.array(xgb_label_list)

# 将数据整形为(N, 6)，以便传递给LightGBM
data = data.reshape(len(data), -1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# 创建LightGBM数据集
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# 定义LightGBM参数
params = {
    'objective': 'binary',  # 二分类问题
    'metric': 'binary_error',  # 评估指标为二分类错误率
    'boosting_type': 'gbdt',  # 使用梯度提升树
    'num_leaves': 128,
    'learning_rate': 0.05,
    'feature_fraction': 0.3
}

# 训练LightGBM模型
num_round = 100  # 迭代次数
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if x >= 0.2 else 0 for x in y_pred]

# 评估模型

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy * 100:.2f}%')

precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

[LightGBM] [Info] Number of positive: 447, number of negative: 2989
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.498166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3610
[LightGBM] [Info] Number of data points in the train set: 3436, number of used features: 470
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.130093 -> initscore=-1.900136
[LightGBM] [Info] Start training from score -1.900136
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

## pattern

In [92]:
ptn_data = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"feats_with_times_{0}.npy"))
ptn_label = np.load(os.path.join(CONFIG.PATH_PROCESSED, f"labels_{0}.npy"))

ptn_data = 2/(1+np.exp(-ptn_data))-1

In [93]:
ptn_data[45]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

## xgboost

In [55]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假设你有N个样本，每个样本是一个3x2的矩阵
# N = 1000
data = np.array(xgb_data_list)
labels = np.array(xgb_label_list)

# 将数据整形为(N, 6)，以便传递给LightGBM
data = data.reshape(len(data), -1)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# model = xgb.XGBClassifier(max_depth=6,
#     learning_rate=0.01,
#     n_estimators=100,
#     objective='binary:logistic',
#     eval_metric='logloss',
#     random_state=42)
model = xgb.XGBClassifier(objective='binary:logistic')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 评估模型

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

ModuleNotFoundError: No module named 'xgboost'

## Random Forest 

In [132]:
from sklearn.ensemble import RandomForestClassifier


data = np.array(xgb_data_list)
labels = np.array(xgb_label_list)

data = data.reshape(len(data), -1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=50, 
                            random_state=42,
                            criterion="gini")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

Accuracy: 84.42%
Precision: 0.3972602739726027
Recall: 0.24369747899159663


# test

In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 创建示例数据（请根据您的数据替换这部分代码）
N = 1000
data = np.random.rand(N, 1, 32, 16)
labels = np.random.randint(0, 2, N)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# 将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 定义基本的ResNet块
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

# 定义ResNet网络
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 创建ResNet模型
def ResNetCustom():
    return ResNet(BasicBlock, [2, 2, 2])  # 这里定义了一个3层的ResNet

# 训练模型
model = ResNetCustom()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 模型评估
model.eval()
y_pred = []
with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_train, y_pred)
print(f"训练准确率: {accuracy:.2f}")


训练准确率: 0.51


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 生成模拟数据
N = 1000
data = np.random.randn(N, 1, 64, 32).astype(np.float32)
labels = np.random.randint(0, 2, N)  # 生成随机标签
data = np.load(os.path.join(CONFIG.PATH_PROCESSED, "feats_64x32_with_times.npy")).astype(np.float32)
labels = np.load(os.path.join(CONFIG.PATH_PROCESSED, "labels_64x32.npy"))

# 划分训练集和测试集
split_ratio = 0.8
split = int(N * split_ratio)
train_data, test_data = data[:split], data[split:]
train_labels, test_labels = labels[:split], labels[split:]

# 创建自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return sample

# 创建数据集和数据加载器
batch_size = 64
train_dataset = CustomDataset(train_data, train_labels)
test_dataset = CustomDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义一个两层CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(32 * 16 * 8, 2)  # 输出2个类别

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 创建模型实例
model = SimpleCNN()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        inputs, labels = batch['data'], batch['label']
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

# 在测试集上验证模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch['data'], batch['label']
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {(correct / total) * 100:.2f}%')


Epoch 1/10, Loss: 0.7166968778922007
Epoch 2/10, Loss: 0.2823257944904841
Epoch 3/10, Loss: 0.1604343234346463
Epoch 4/10, Loss: 0.148019632181296
Epoch 5/10, Loss: 0.13936221699875134
Epoch 6/10, Loss: 0.11386105848046449
Epoch 7/10, Loss: 0.10020001146655816
Epoch 8/10, Loss: 0.09341820701956749
Epoch 9/10, Loss: 0.0867923440841528
Epoch 10/10, Loss: 0.0762054229585024
Accuracy on test set: 95.22%


In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import timm

# 步骤1: 生成数据集
class CustomDataset(Dataset):
    def __init__(self, num_samples):
        self.num_samples = num_samples
        self.data = torch.randn(num_samples, 1, 32, 16)  # 随机生成示例数据
        self.labels = torch.randint(0, 2, (num_samples,))  # 随机生成示例标签 (0或1)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 步骤2: 定义 ViT 模型
class ViTClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(ViTClassifier, self).__init__()

        self.model = timm.create_model(model_name, pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

# 步骤3: 创建数据加载器
num_samples = 1000  # 你的数据集大小
batch_size = 32

dataset = CustomDataset(num_samples)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 步骤4: 定义损失函数和优化器
model_name = 'vit_base_patch16_224'  # 选择 ViT 模型架构
num_classes = 2  # 你的分类任务中的类别数

model = ViTClassifier(model_name, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 步骤5: 训练模型
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

print("Training complete")

# 步骤6: 进行推理
# 为了进行推理，可以加载保存的模型权重并使用模型进行分类预测

# 保存模型权重
torch.save(model.state_dict(), 'vit_model.pth')

# 加载模型权重
model.load_state_dict(torch.load('vit_model.pth'))
model.eval()  # 设置模型为评估模式

# 准备一个示例输入（假设你有一个图像 tensor）
input_image = torch.randn(1, 1, 32, 16)  # 示例输入

# 使用模型进行图像分类预测
with torch.no_grad():
    output = model(input_image)

# 输出的概率分布用于分类预测
predicted_class = torch.argmax(output, dim=1)
print("Predicted Class:", predicted_class.item())


2023-09-16 01:09:55,109 - INFO - Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_224.augreg2_in21k_ft_in1k)


LocalEntryNotFoundError: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.

In [71]:
from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTConfig

model_name = "google/vit-base-patch16-224-in21k"  # 选择 ViT 模型名称

# 获取模型配置（不下载权重）
model_config = ViTConfig.from_pretrained(model_name)

# 打印模型配置
print(model_config)

ModuleNotFoundError: No module named 'transformers'

In [69]:
!pip install transformers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import numpy as np

# 定义数据生成器
def generate_data(N):
    data = np.random.rand(N, 1, 32, 16).astype(np.float32)
    labels = np.random.randint(2, size=N)
    return torch.tensor(data), torch.tensor(labels)

# 定义CNN模型类
class CNNModel(nn.Module):
    def __init__(self, num_classes, conv1_kernel_size, conv2_kernel_size):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=conv1_kernel_size, padding=(conv1_kernel_size-1)//2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=conv2_kernel_size, padding=(conv2_kernel_size-1)//2)
        self.fc1 = nn.Linear(64 * 8 * 4, 128)  # 根据数据大小进行调整
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 8 * 4)  # 根据数据大小进行调整
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义训练函数
def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# 定义评估函数
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 定义Optuna试验函数
def objective(trial):
    # 定义超参数搜索范围
    conv1_kernel_size = trial.suggest_int("conv1_kernel_size", 3, 7)
    conv2_kernel_size = trial.suggest_int("conv2_kernel_size", 1, 3)
    
    # 创建数据加载器
    train_data, train_labels = generate_data(1000)
    train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    val_data, val_labels = generate_data(200)
    val_dataset = torch.utils.data.TensorDataset(val_data, val_labels)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # 创建模型
    model = CNNModel(num_classes=2, conv1_kernel_size=conv1_kernel_size, conv2_kernel_size=conv2_kernel_size)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 训练模型
    train(model, train_loader, criterion, optimizer, num_epochs=5)
    
    # 评估模型
    accuracy = evaluate(model, val_loader)
    
    return -accuracy  # 我们希望最大化召回率，因此返回负的准确度

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)  # 运行10次试验
    best_params = study.best_params
    best_accuracy = -study.best_value
    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy: {best_accuracy}")


[I 2023-09-18 13:03:47,759] A new study created in memory with name: no-name-c65ffe4a-5346-4c2e-a97f-119b6a575c7a
[W 2023-09-18 13:03:47,791] Trial 0 failed with parameters: {'conv1_kernel_size': 5, 'conv2_kernel_size': 2} because of the following error: ValueError('Expected input batch_size (21) to match target batch_size (32).').
Traceback (most recent call last):
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2229641/2070025995.py", line 79, in objective
    train(model, train_loader, criterion, optimizer, num_epochs=5)
  File "/tmp/ipykernel_2229641/2070025995.py", line 39, in train
    loss = criterion(outputs, labels)
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/anichikage/anaconda3/envs/dram-py310/

ValueError: Expected input batch_size (21) to match target batch_size (32).

In [83]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import numpy as np

# 定义数据生成器
def generate_data(N):
    data = np.random.rand(N, 1, 32, 16).astype(np.float32)
    labels = np.random.randint(2, size=N)
    return torch.tensor(data), torch.tensor(labels)

# 定义CNN模型类
class CNNModel(nn.Module):
    def __init__(self, num_classes, conv1_kernel_size, conv2_kernel_size):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=conv1_kernel_size, padding=(conv1_kernel_size-1)//2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=conv2_kernel_size, padding=(conv2_kernel_size-1)//2)
        self.fc1 = nn.Linear(64 * 8 * 4, 128)  # 根据数据大小进行调整
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 8 * 4)  # 根据数据大小进行调整
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义训练函数
def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# 定义评估函数
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 定义Optuna试验函数
def objective(trial):
    # 定义超参数搜索范围
    conv1_kernel_size = trial.suggest_int("conv1_kernel_size", 3, 5, step=2)
    conv2_kernel_size = trial.suggest_int("conv2_kernel_size", 1, 3, step=2)
    
    # 创建数据加载器
    train_data, train_labels = generate_data(1000)
    train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    val_data, val_labels = generate_data(200)
    val_dataset = torch.utils.data.TensorDataset(val_data, val_labels)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # 创建模型
    model = CNNModel(num_classes=2, conv1_kernel_size=conv1_kernel_size, conv2_kernel_size=conv2_kernel_size)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 训练模型
    train(model, train_loader, criterion, optimizer, num_epochs=5)
    
    # 评估模型
    accuracy = evaluate(model, val_loader)
    
    return -accuracy  # 我们希望最大化召回率，因此返回负的准确度

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)  # 运行10次试验
    best_params = study.best_params
    best_accuracy = -study.best_value
    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy: {best_accuracy}")


[I 2023-09-18 13:07:31,039] A new study created in memory with name: no-name-d012f223-12b6-4bb8-81d5-64f2199e582a
[I 2023-09-18 13:07:31,809] Trial 0 finished with value: -0.48 and parameters: {'conv1_kernel_size': 3, 'conv2_kernel_size': 1}. Best is trial 0 with value: -0.48.
[I 2023-09-18 13:07:32,653] Trial 1 finished with value: -0.5 and parameters: {'conv1_kernel_size': 5, 'conv2_kernel_size': 3}. Best is trial 0 with value: -0.48.
[I 2023-09-18 13:07:33,415] Trial 2 finished with value: -0.555 and parameters: {'conv1_kernel_size': 3, 'conv2_kernel_size': 1}. Best is trial 0 with value: -0.48.
[I 2023-09-18 13:07:34,159] Trial 3 finished with value: -0.54 and parameters: {'conv1_kernel_size': 5, 'conv2_kernel_size': 1}. Best is trial 0 with value: -0.48.
[I 2023-09-18 13:07:34,988] Trial 4 finished with value: -0.505 and parameters: {'conv1_kernel_size': 3, 'conv2_kernel_size': 3}. Best is trial 0 with value: -0.48.
[I 2023-09-18 13:07:35,754] Trial 5 finished with value: -0.565 a

Best Parameters: {'conv1_kernel_size': 3, 'conv2_kernel_size': 1}
Best Accuracy: 0.43


## transfomer

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 准备数据
N = 1000  # 假设有1000个样本
image_size = (1, 32, 16)
num_classes = 2
data = torch.randn(N, *image_size)
labels = torch.randint(0, num_classes, (N,))

# 划分训练集和测试集
split_ratio = 0.8
split_idx = int(N * split_ratio)
train_data, test_data = data[:split_idx], data[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]

# 创建数据加载器
batch_size = 32
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 创建ViT模型
class ViT(nn.Module):
    def __init__(self, image_size, num_classes, patch_size, hidden_dim, num_heads, num_layers):
        super(ViT, self).__init__()
        self.embedding = nn.Conv2d(image_size[0], hidden_dim, kernel_size=patch_size, stride=patch_size)
        self.transformer = nn.Transformer(
            d_model=hidden_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), x.size(1), -1)  # Flatten spatial dimensions
        x = x.permute(2, 0, 1)  # Adjust the dimension order
        x = self.transformer(x, x)  # Self-attention with source and target being the same
        x = x.mean(dim=0)  # Global average pooling
        x = self.fc(x)
        return x

# 创建ViT模型实例
model = ViT(image_size, num_classes, patch_size=(8, 8), hidden_dim=64, num_heads=4, num_layers=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("当前设备:", device)
model.to(device)


# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        data_gpu = data.to(device)
        labels_gpu = labels.to(device)
        outputs = model(data_gpu)
        loss = criterion(outputs, labels_gpu)
        loss.backward()
        optimizer.step()
        

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

# 在测试集上评估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        data_gpu = data.to(device)
        labels_gpu = labels.to(device)
        outputs = model(data_gpu)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels_gpu).sum().item()
        
        curdata = np.array(data)
        print(curdata.shape)
        

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')


当前设备: cuda
Epoch [1/1], Loss: 0.6972571015357971
(32, 1, 32, 16)
(32, 1, 32, 16)
(32, 1, 32, 16)
(32, 1, 32, 16)
(32, 1, 32, 16)
(32, 1, 32, 16)
(8, 1, 32, 16)
Test Accuracy: 47.0%


In [18]:
import numpy as np

# 创建一个示例的二维NumPy矩阵
matrix = np.array([[1, 2, 3],
                  [-1, 0, 2],
                  [0, 0, 0],
                  [4, 5, 6]])

# 计算每行的和
row_sums = np.sum(matrix, axis=1)

# 使用向量化操作检查每行的和是否大于0，得到一个布尔值的矩阵
row_sum_greater_than_zero = row_sums > 0

# 统计满足条件的行数
count = np.sum(row_sum_greater_than_zero)

print(f"满足条件的 行数：{count}")


满足条件的行数：3


In [20]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cuda', index=0)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 准备数据
N = 1000  # 假设有1000个样本
image_size = (1, 32, 16)
num_classes = 2
data = torch.randn(N, *image_size)
labels = torch.randint(0, num_classes, (N,))

# 划分训练集和测试集
split_ratio = 0.8
split_idx = int(N * split_ratio)
train_data, test_data = data[:split_idx], data[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]

# 创建数据加载器
batch_size = 32
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 创建ViT模型
class ViT(nn.Module):
    def __init__(self, image_size, num_classes, patch_size, hidden_dim, num_heads, num_layers):
        super(ViT, self).__init__()
        self.embedding = nn.Conv2d(image_size[0], hidden_dim, kernel_size=patch_size, stride=patch_size)
        self.transformer = nn.Transformer(
            d_model=hidden_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), x.size(1), -1)  # Flatten spatial dimensions
        x = x.permute(2, 0, 1)  # Adjust the dimension order
        x = self.transformer(x, x)  # Self-attention with source and target being the same
        x = x.mean(dim=0)  # Global average pooling
        x = self.fc(x)
        return x

# 创建ViT模型实例
model = ViT(image_size, num_classes, patch_size=(8, 8), hidden_dim=64, num_heads=4, num_layers=4)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("当前设备:", device)
model.to(device)


# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        data_gpu = data.to(device)
        labels_gpu = labels.to(device)
        outputs = model(data_gpu)
        loss = criterion(outputs, labels_gpu)
        loss.backward()
        optimizer.step()
        # print(f"{loss}")

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

# 在测试集上评估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        data_gpu = data.to(device)
        labels_gpu = labels.to(device)
        outputs = model(data_gpu)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels_gpu).sum().item()
        
        curdata = np.array(data)
        # print(curdata.shape)
        

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')


当前设备: cuda:1
Epoch [1/1], Loss: 0.7072823643684387
Test Accuracy: 44.0%


In [2]:
import numpy as np

# 创建一个示例的二维NumPy矩阵
matrix = np.array([[0, 0, 0],
                  [1, 2, 3],
                  [0, 0, 0],
                  [4, 5, 6],
                  [0, 0, 0]])

# 找到非零元素的行索引
non_zero_rows = np.any(matrix != 0, axis=1)

# 找到非零元素的最大和最小行索引
max_non_zero_row_index = np.max(np.where(non_zero_rows))
min_non_zero_row_index = np.min(np.where(non_zero_rows))

# 计算最大和最小行索引之间的行数
row_count_between = max_non_zero_row_index - min_non_zero_row_index + 1

print(f"非零元素的最大行索引：{max_non_zero_row_index}")
print(f"非零元素的最小行索引：{min_non_zero_row_index}")
print(f"行数之间的行数：{row_count_between}")


非零元素的最大行索引：3
非零元素的最小行索引：1
行数之间的行数：3


In [3]:
import numpy as np

# 创建一个示例的二维NumPy矩阵
matrix = np.array([[0, 0, 0, 1, 0],
                  [0, 0, 2, 0, 0],
                  [0, 0, 0, 0, 0],
                  [4, 5, 0, 0, 0],
                  [0, 0, 0, 0, 0]])

# 找到非零元素的行索引和列索引
non_zero_rows, non_zero_cols = np.nonzero(matrix)

# 找到非零元素的最大行索引和最小列索引
max_non_zero_row_index = np.max(non_zero_rows)
min_non_zero_col_index = np.min(non_zero_cols)

# 计算最小列索引与最大列索引之间的列数
column_count_between = max_non_zero_row_index - min_non_zero_col_index + 1

print(f"非零元素的最大行索引：{max_non_zero_row_index}")
print(f"非零元素的最小列索引：{min_non_zero_col_index}")
print(f"列数之间的列数：{column_count_between}")


非零元素的最大行索引：3
非零元素的最小列索引：0
列数之间的列数：4


In [6]:
matrix = np.array([2,3,4])

In [7]:
len(matrix)

3

## cnn

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from gradcam import GradCAM
import matplotlib.pyplot as plt
import numpy as np

import warnings
import logging
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

N=10000
M=100
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(32 * 8 * 4, 2)  # 32 * 8 * 4 is the output size after 2 conv layers

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 4)
        x = self.fc(x)
        return x

# Load and preprocess your data
# Assuming you have your data as tensors X_train, y_train, X_test, and y_test
# Convert them into PyTorch tensors and datasets

X_train = torch.randn(N, 1, 32, 16)
y_train = torch.randint(2, (N,))
X_test = torch.randn(M, 1, 32, 16)
y_test = torch.randint(2, (M,))

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
model.to("cuda")

model = nn.DataParallel(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputss, labelss = inputs.to("cuda"),labels.to("cuda") 
        optimizer.zero_grad()
        outputs = model(inputss)
        loss = criterion(outputs, labelss)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Test the model
model.eval()
correct = 0
total = 0

target_layer = model.conv1  # Assuming 'conv2' is the layer you want to visualize
gradcam = GradCAM(model, target_layer)
# gradcam = GradCAM(model, target_layer='conv2')  # Replace 'conv2' with the layer you want to visualize

for inputs, labels in test_loader:
    inputss, labelss = inputs.to("cuda"),labels.to("cuda") 
    outputs = model(inputss)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labelss).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

model.to("cpu")
positive_heatmaps = []
negative_heatmaps = []
for inputs, labels in test_loader:
    # inputss, labelss = inputs.to("cuda"),labels.to("cuda") 
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

    for i in range(inputs.size(0)):
        if predicted[i] == labels[i]:
            # Correctly classified samples
            heatmap, _ = gradcam(inputs[i].unsqueeze(0))
            heatmap = heatmap.squeeze().numpy()  # Extract the heatmap and convert to NumPy array
            if predicted[i] == 1:
                positive_heatmaps.append(heatmap)
            else:
                negative_heatmaps.append(heatmap)
average_positive_heatmap = np.mean(positive_heatmaps, axis=0)
average_negative_heatmap = np.mean(negative_heatmaps, axis=0)
heatmap_difference = average_positive_heatmap - average_negative_heatmap
threshold = 0.1  # 设定差异阈值，根据实际情况调整
heatmap_difference[heatmap_difference < threshold] = 0

print(average_positive_heatmap)

average_negative_heatmap = average_negative_heatmap / 10000
print(average_negative_heatmap)

# 可视化差异区域
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(average_positive_heatmap, cmap='jet')
plt.title('Average Grad-CAM for Correct Positive Samples')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(average_negative_heatmap, cmap='jet')
plt.title('Average Grad-CAM for Correct Negative Samples')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(heatmap_difference, cmap='jet')
plt.title('Heatmap Difference')
plt.axis('off')
        #     # Plot the heatmap
        #     plt.figure(figsize=(6, 6))
        #     plt.imshow(heatmap, cmap='jet')
        #     plt.title(f'Correct: Predicted={predicted[i]}, Actual={labels[i]}')
        #     plt.axis('off')
        #     plt.show()
        # else:
        #     # Incorrectly classified samples (if needed)
        #     pass

2023-09-22 15:48:08,297 - INFO - Epoch 1/100, Loss: 0.6953883015426101
2023-09-22 15:48:08,990 - INFO - Epoch 2/100, Loss: 0.6916598795325892
2023-09-22 15:48:09,691 - INFO - Epoch 3/100, Loss: 0.6874425490950323
2023-09-22 15:48:10,379 - INFO - Epoch 4/100, Loss: 0.682596045314886
2023-09-22 15:48:11,076 - INFO - Epoch 5/100, Loss: 0.6756010940120478
2023-09-22 15:48:11,770 - INFO - Epoch 6/100, Loss: 0.663198435762126
2023-09-22 15:48:12,477 - INFO - Epoch 7/100, Loss: 0.6513456436479168
2023-09-22 15:48:13,184 - INFO - Epoch 8/100, Loss: 0.6371911600896507
2023-09-22 15:48:13,874 - INFO - Epoch 9/100, Loss: 0.6248618831300432
2023-09-22 15:48:14,574 - INFO - Epoch 10/100, Loss: 0.6107565339203853
2023-09-22 15:48:15,289 - INFO - Epoch 11/100, Loss: 0.5968862639111319
2023-09-22 15:48:15,995 - INFO - Epoch 12/100, Loss: 0.5774357652967903
2023-09-22 15:48:16,761 - INFO - Epoch 13/100, Loss: 0.5662180661775504
2023-09-22 15:48:17,523 - INFO - Epoch 14/100, Loss: 0.5551156170049291
202

AttributeError: 'DataParallel' object has no attribute 'conv1'

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 6 * 3, 128)
        self.fc2 = nn.Linear(128, 1)  # 1 output for binary classification

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Generate sample data and labels (replace with your own data)
N = 100  # Replace with your actual data size
data = np.random.randn(N, 1, 32, 16)
labels = np.random.randint(0, 2, N)

# Split data into training and testing sets
split_ratio = 0.8
split_idx = int(N * split_ratio)
train_data, test_data = data[:split_idx], data[split_idx:]
train_labels, test_labels = labels[:split_idx], labels[split_idx:]

# Create a DataLoader (you might need to adjust batch size and other parameters)
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    list(zip(train_data, train_labels)),
    batch_size=batch_size,
    shuffle=True,
)

# Initialize the CNN model, loss function, and optimizer
model = CNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (adjust as needed)
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float()).squeeze()  # Squeeze the output to match the labels' shape
        loss = criterion(outputs, labels.float())  # Use BCEWithLogitsLoss for binary classification
        loss.backward()
        optimizer.step()

# Evaluation on the test set
test_data = torch.tensor(test_data).float()
with torch.no_grad():
    model.eval()
    outputs = model(test_data).squeeze()  # Squeeze the output to match the labels' shape
    predicted_labels = (torch.sigmoid(outputs) > 0.5).float()  # Convert logits to binary predictions

# Extract feature maps for correctly classified samples
correct_samples = test_data[(predicted_labels == test_labels).bool()]
correct_labels = test_labels[(predicted_labels == test_labels).bool()]

# Define a hook to extract feature maps from the second convolutional layer
feature_maps = []

def hook_fn(module, input, output):
    feature_maps.append(output)

hook = model.conv2.register_forward_hook(hook_fn)

# Forward pass for correctly classified samples to extract feature maps
_ = model(correct_samples)

# Remove the hook
hook.remove()

# Apply t-SNE to the feature maps
feature_maps = torch.cat(feature_maps, dim=0)
feature_maps = feature_maps.view(feature_maps.size(0), -1)  # Flatten feature maps

tsne = TSNE(n_components=2)
tsne_result = tsne.fit_transform(feature_maps.numpy())  # Convert to NumPy array for t-SNE

# Separate positive and negative samples based on labels
positive_indices = (correct_labels == 1).bool()
negative_indices = (correct_labels == 0).bool()

# Plot t-SNE results
plt.figure(figsize=(10, 6))
plt.scatter(tsne_result[positive_indices, 0], tsne_result[positive_indices, 1], c='g', label='Positive')
plt.scatter(tsne_result[negative_indices, 0], tsne_result[negative_indices, 1], c='r', label='Negative')
plt.title("t-SNE Visualization of Correctly Classified Sample Feature Maps")
plt.legend()
plt.show()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x384 and 576x128)

In [42]:
import torch
import torch.nn as nn

class TwoLayerCNN(nn.Module):
    def __init__(self, in_channels, out_channels1, out_channels2, kernel_size1, kernel_size2):
        super(TwoLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels1, kernel_size=kernel_size1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels1, out_channels2, kernel_size=kernel_size2)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(out_channels2, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

# 指定模型参数
in_channels = 1  # 输入通道数
out_channels1 = 16  # 第一层卷积核数量
out_channels2 = 32  # 第二层卷积核数量
kernel_size1 = 3  # 第一层卷积核大小
kernel_size2 = 3  # 第二层卷积核大小
num_classes = 10  # 分类类别数量

# 创建模型实例
model = TwoLayerCNN(in_channels, out_channels1, out_channels2, kernel_size1, kernel_size2)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型（假设有训练数据和标签，以下是伪代码）
# for epoch in range(num_epochs):
#     for batch in train_loader:
#         data, labels = batch
#         optimizer.zero_grad()
#         outputs = model(data)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

# 保存模型参数到文件
torch.save(model.state_dict(), 'two_layer_cnn_model.pth')

# 加载模型
loaded_model = TwoLayerCNN(in_channels, out_channels1, out_channels2, kernel_size1, kernel_size2)
loaded_model.load_state_dict(torch.load('two_layer_cnn_model.pth'))
loaded_model.eval()  # 进入评估模式，用于推断


TwoLayerCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (fc): Linear(in_features=32, out_features=10, bias=True)
)

In [44]:
import re

# 目标字符串
target_string = "../data/processed/agg/Server_22848_1_feats_32x16_11.npy"

# 定义正则表达式模式，匹配"Server_"后面跟着数字的部分
pattern = r"Server_(\d+)"

# 使用正则表达式查找匹配的内容
match = re.search(pattern, target_string)

# 提取匹配的部分
if match:
    extracted_string = match.group(0)

    # 输出提取的字符串
    print(extracted_string)
else:
    print("未找到匹配的字符串")


Server_22848
